In [115]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random
import pickle
import time
import datetime
import xlsxwriter
import circuit_builder
import importlib
importlib.reload(circuit_builder)
import scipy.io as spio


#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

backend = "local_qasm_simulator"


# prepare a state of all zeros
def prepareZeros(qc):
    qc.h(qr[1])
    qc.barrier()
    qc.cx(qr[1],qr[0])
    qc.barrier()
    qc.cx(qr[1],qr[2])
    qc.barrier()
    qc.cx(qr[2],qr[3])
    qc.barrier()
    
def setInput(qc, compiler):
    if compiler == 0: 
        return
    elif compiler == 1:
        qc.x(qr[0])
        qc.x(qr[2])
    elif compiler == 2:
        qc.x(qr[0])
        qc.x(qr[1])
    elif compiler == 3:
        qc.x(qr[2])
        qc.x(qr[1])
    qc.barrier()
        
'''
def setInput(qc,compiler):
    if compiler == 0:
        return
    elif compiler == 1:
        qc.x(qr[1])
    elif compiler == 2:
        qc.x(qr[0])
    elif compiler == 3:
        qc.x(qr)
            '''
    
shots = 1024

# code for extracting gate constructions
extract = spio.loadmat('somegates.mat', squeeze_me = True)
CZ = extract['CZ12py']
H1 = extract['H1py']

# specify circuits
circuits = H1
#circuits = [["Permute 2 1", "X 1", "Permute 2 1"]]
reps = 1

# fill these in to get a calculation of success rate
initialStates = [0,1,2,3]

# these depend on the code
desiredOutputs = [["10"], ["00"], ["11"], ["01"]]
validOutputs = ["00", "01", "10", "11"]

    
# main benchmarking procedure
for circuit in circuits:
    for compiler in initialStates:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        for trial in range(reps):
            ## this depends on the code
            n = 4
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            ## this depends on the code
            prepareZeros(qc)
            
            ## this depends on the code
            # maps the input info into actual physical qubits
            perm = {1: 3, 2:2, 3:1, 4:0 }
        
            
            # sets our starting state to 00, 10, 01, or 11
            setInput(qc, compiler)
            
            # do what we want
            circuit_builder.buildCirc(qc, qr, cr, circuit, perm)

            job = execute(qc, backend,shots=shots)
            result = job.result()
            data = result.get_counts(qc)
            
            # comment this out if don't care about the individual data
            print(data)
            
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value

            if totalValid == 0:
                print("oops")
            else:
                # comment out if running more trials
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")


Qconfig loaded from C:\Users\ddrsq\OneDrive\Quantum\qecc_benchmark\Qconfig.py.

Circuit is: ['Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2' 'CNOT 1  4'
 'H 1  2  3  4' 'CZ 1  4' 'CZ 2  4' 'H 1  2  3' 'Permute 1  4  2  3'
 'CNOT 1  2']
Compiler is: 0
{'0000': 233, '0011': 278, '1100': 254, '1111': 259}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2' 'CNOT 1  4'
 'H 1  2  3  4' 'CZ 1  4' 'CZ 2  4' 'H 1  2  3' 'Permute 1  4  2  3'
 'CNOT 1  2']
Compiler is: 1
{'0000': 265, '0011': 254, '1100': 238, '1111': 267}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2' 'CNOT 1  4'
 'H 1  2  3  4' 'CZ 1  4' 'CZ 2  4' 'H 1  2  3' 'Permute 1  4  2  3'
 'CNOT 1  2']
Compiler is: 2
{'0101': 248, '0110': 267, '1001': 245, '1010': 264}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 2  1  4  3' 'CNOT 4  2' 'CNOT 2  4' 'CNOT 1  2' 'CNOT 1  4'
 'H 1  2  3  4' 'CZ 1 

{'0011': 252, '0101': 238, '1010': 264, '1100': 270}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 1  4  2  3' 'CNOT 2  1' 'CNOT 4  1' 'CNOT 3  4' 'CNOT 2  3'
 'CNOT 2  4' 'CNOT 1  3' 'CNOT 1  4' 'H 1  2  3  4' 'P 1  2  3' 'CZ 1  2'
 'CZ 1  3' 'CZ 1  4' 'CZ 2  3' 'CZ 2  4' 'H 1  2  3' 'Permute 1  4  2  3'
 'CNOT 1  2']
Compiler is: 3
{'0011': 242, '0101': 279, '1010': 254, '1100': 249}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 1  4  2  3' 'CNOT 2  1' 'CNOT 4  1' 'CNOT 2  3' 'CNOT 2  4'
 'CNOT 1  3' 'H 1  2  3  4' 'P 1  2  3  4' 'CZ 1  2' 'CZ 1  3' 'CZ 1  4'
 'CZ 2  3' 'CZ 2  4' 'H 1  2  3' 'P 2  3' 'CZ 2  3' 'Permute 1  4  2  3'
 'CNOT 1  2' 'X 1  3']
Compiler is: 0
{'0000': 273, '0110': 244, '1001': 242, '1111': 265}
oops
Average of success rates is: 0.0

Circuit is: ['Permute 1  4  2  3' 'CNOT 2  1' 'CNOT 4  1' 'CNOT 2  3' 'CNOT 2  4'
 'CNOT 1  3' 'H 1  2  3  4' 'P 1  2  3  4' 'CZ 1  2' 'CZ 1  3' 'CZ 1  4'
 'CZ 2  3' 'CZ 2  4' 'H 1  2  3' 'P 2  3' 'C